<h1 style="font-size:50px; text-align:center">DCO Detections</h1>

In [101]:
import h5py as h5
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import seaborn as sns
import legwork
import os
import sys
sys.path.append("../src/")
from variations import variations
from galaxy import simulate_mw

In [102]:
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='serif')
fs = 20

params = {'legend.fontsize': fs,
         'axes.labelsize': fs,
         'xtick.labelsize':0.7*fs,
         'ytick.labelsize':0.7*fs}
plt.rcParams.update(params)

In [103]:
dco_colours = {"BHBH": plt.get_cmap("plasma")(0.2), 
               "BHNS": plt.get_cmap("plasma")(0.5),
               "NSNS": plt.get_cmap("plasma")(0.8)}

dco_types = ["BHBH", "BHNS", "NSNS"]
data_path = "../data/"
sim_folder = data_path + "simulation/"  #"../data/old_sims/simulation_plus_supp/"

total_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW.npy")
total_bound_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW_nohubble.npy")

# Calculate number of detections

In [105]:
def get_detections(dco_type, MW_SIZE=100000, MW_MASS_FAC=1.5, t_obs=4*u.yr, only_frac=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))
    
    # go through each physics variation
#     for d in range(len(dco_types)):
    
    for d in range(len(dco_types)):
        for v in range(len(variations)):
            # open the proper output file
            fname = sim_folder + "{}_{}_all.h5".format(dco_types[d], variations[v]["file"])
            if os.path.isfile(fname):
                with h5.File(fname, "r") as f:
                    n_ten_year = f["simulation"].attrs["n_ten_year"].astype(np.int)
                    total_mw_weight = f["simulation"].attrs["total_MW_weight"]
                    full_data = f["simulation"][...].squeeze()
                
#                 circ_sources = legwork.source.Source(m_1=full_data["m_1"] * u.Msun, m_2=full_data["m_2"] * u.Msun,
#                                            dist=full_data["dist"] * u.kpc, ecc=np.zeros(len(full_data["m_1"])),
#                                            a=full_data["a_LISA"] * u.AU)
#                 circ_snr = circ_sources.get_snr()
                
#                 print(dco_types[d])
#                 true_rate = np.sum(full_data["weight"][full_data["snr"] >= 7]) / np.sum(total_mw_weight) * total_DCOs_in_MW[d][v] * MW_MASS_FAC
#                 circ_rate = np.sum(full_data["weight"][circ_snr >= 7]) / np.sum(total_mw_weight) * total_DCOs_in_MW[d][v] * MW_MASS_FAC
#                 print(true_rate, circ_rate, (true_rate - circ_rate) / true_rate)
#                 print()

                # go through the file and normalise the detections for all binaries with SNR > x
                cursor = 0
                detections_per_MW = np.zeros(len(n_ten_year))
                
                checks = np.zeros(len(n_ten_year))
                for i in range(len(n_ten_year)):
                    snr = full_data["snr"][cursor:cursor + n_ten_year[i]]
                    weights = full_data["weight"][cursor:cursor + n_ten_year[i]]
                    checks = np.sum(weights)**2 / np.sum(weights**2)
                    detections[d][v][i] = np.sum(weights[snr * np.sqrt(t_obs / (4 * u.yr)) > 7]) / total_mw_weight[i]
                    cursor += n_ten_year[i]
                if not only_frac:
                    detections[d][v] *= total_DCOs_in_MW[d][v] * MW_MASS_FAC

            # set to zero if no file exists (simulation will have crashed)
            else:
                detections[d][v] = 0
    
    return detections

In [5]:
def get_detections_alt(dco_type, MW_SIZE=100000, MW_MASS_FAC=1.5, t_obs=4*u.yr, only_frac=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))
    
    fname = sim_folder + "{}_{}_all.h5".format(dco_types[dco_type], variations[0]["file"])
    if os.path.isfile(fname):
        with h5.File(fname, "r") as f:
            total_mw_weight = f["simulation"].attrs["total_MW_weight"].sum()
            full_data = f["simulation"][...].squeeze()
            
        factor = total_DCOs_in_MW[dco_type][0] * MW_MASS_FAC / total_mw_weight
        
        weights = full_data["weight"][full_data["snr"] > 7]
        avg = weights.sum() * factor
        print(avg)
        
    sample_rates = np.zeros(1000)
    for i in range(len(sample_rates)):
        sample = np.random.choice(weights, len(weights), replace=True)
        sample_rates[i] = sample.sum() * factor
        
    print(np.percentile(sample_rates, [5, 95]))
    
    sample_rates = np.zeros(1000)
    for i in range(len(sample_rates)):
        sample = np.random.choice(weights, 100, replace=True)
        sample_rates[i] = sample.sum() * factor * (len(weights) / 100)
        
    print(np.percentile(sample_rates, [5, 95]))

In [106]:
detections_4yr = get_detections(dco_types)

In [76]:
detections_10yr = get_detections(dco_types, t_obs=10*u.yr)

35.70766953242095
4.6358057473799406
32.84556507740563
11.434237587159709
8.358122307523175
14.564530957325884


In [77]:
detections_only_fractions = get_detections(dco_types, only_frac=True)

35.70766953242095
4.6358057473799406
32.84556507740563
11.434237587159709
8.358122307523175
14.564530957325884


In [107]:
np.save("../data/detections_4yr", detections_4yr)
np.save("../data/detections_10yr", detections_10yr)
np.save("../data/detections_only_fractions", detections_only_fractions)

# What if we assumed everything was circular?

In [91]:
fid_sources = [None, None, None]
for i in range(len(dco_types)):
    with h5.File(sim_folder + "{}_fiducial_all.h5".format(dco_types[i]), "r") as f:
        full_data = f["simulation"][...].squeeze()
        snr_mask = full_data["snr"] > 7
        
        data = full_data[snr_mask]
        
    fid_sources[i] = legwork.source.Source(m_1=data["m_1"] * u.Msun, m_2=data["m_2"] * u.Msun,
                                           dist=data["dist"] * u.kpc, ecc=data["e_LISA"],
                                           a=data["a_LISA"] * u.AU)
    fid_sources[i].weight = data["weight"]

In [92]:
for i in range(len(dco_types)):
    snr_true = fid_sources[i].get_snr(verbose=True)
    new_source = fid_sources[i]
    new_source.ecc = np.zeros(new_source.n_sources)
    snr_circ = new_source.get_snr(verbose=True)
    
    print(np.sum(new_source.weight[snr_circ > 7]) / np.sum(new_source.weight))

Calculating SNR for 46424 sources
	46421 sources are stationary
		30416 sources are stationary and circular
		16005 sources are stationary and eccentric
	3 sources are evolving
		3 sources are evolving and circular
Calculating SNR for 46424 sources
	46421 sources are stationary
		46421 sources are stationary and circular
	3 sources are evolving
		3 sources are evolving and circular
0.7607342497723641
Calculating SNR for 44927 sources
	44925 sources are stationary
		41843 sources are stationary and circular
		3082 sources are stationary and eccentric
	2 sources are evolving
		2 sources are evolving and circular
Calculating SNR for 44927 sources
	44925 sources are stationary
		44925 sources are stationary and circular
	2 sources are evolving
		2 sources are evolving and circular
0.8955950229445685
Calculating SNR for 32805 sources
	32803 sources are stationary
		18591 sources are stationary and circular
		14212 sources are stationary and eccentric
	2 sources are evolving
		2 sources are 

Calculating SNR for 46424 sources
	46421 sources are stationary
		30416 sources are stationary and circular
		16005 sources are stationary and eccentric
	3 sources are evolving
		3 sources are evolving and circular


Calculating SNR for 46424 sources
	46421 sources are stationary
		46421 sources are stationary and circular
	3 sources are evolving
		3 sources are evolving and circular


In [89]:
len(snr_true[snr_true > 7]) / len(snr_true)

0.9993107013613648

In [90]:
len(snr_circ[snr_circ > 7]) / len(snr_circ)

0.7934904359813889